In [ ]:
import os
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [2]:
import os

#path
os.chdir('/content/drive/Shared drives/Machine Learning/DATASET/INGREDIENTS/')
print("Current working directory:", os.getcwd())

!ls "{os.getcwd()}"

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/Shared drives/Machine Learning/DATASET/INGREDIENTS/'

In [3]:
subfolders = [f for f in os.listdir(os.getcwd()) if os.path.isdir(f)]
print("Subfolders in directory:", subfolders)

example_imgs = []

for subfolder in subfolders:
    subfolder_path = os.path.join(os.getcwd(), subfolder)
    image_files = os.listdir(subfolder_path)

    image_extensions = ('.jpeg', '.jpg', '.png')
    for file in image_files:
        if file.endswith(image_extensions):
            img = Image.open(os.path.join(subfolder_path, file))
            example_imgs.append(img)

num_images = len(example_imgs)
if num_images == 0:
    print("No images found in the specified directories.")
else:
    w, h = example_imgs[0].size
    grid = Image.new('RGBA', size=(3 * w, (num_images // 3 + 1) * h))
    grid_w, grid_h = grid.size
    ls = grid_w / 3

    for i, img in enumerate(example_imgs):
        grid.paste(img, box=(i % 3 * w, i // 3 * h))

    # Menampilkan gambar
    plt.figure(figsize=(18, 10))
    plt.title("Example Images", fontsize=18)
    plt.imshow(grid)
    plt.xticks([ls - ls / 2, ls * 2 - ls / 2, ls * 3 - ls / 2],
               [f"Image {i+1}" for i in range(min(3, num_images))], fontsize=15)
    plt.tick_params(axis=u'both', which=u'both', length=0)
    plt.yticks([])
    plt.show()

Subfolders in directory: ['.git']
No images found in the specified directories.


In [4]:
#from keras.preprocessing import image
from PIL import Image, ImageDraw, ImageFont

ayam_img = Image.open("dagingayam/daging_ayam (239).jpg")

example_imgs = [ayam_img, ayam_img, ayam_img]

w, h = example_imgs[1].size
grid = Image.new('RGBA', size=(3*w, h))
grid_w, grid_h = grid.size
ls = grid_w/3

for i, img in enumerate(example_imgs):
    grid.paste(img, box=(i%3*w, i//3*h))

plt.figure(figsize=(18, 10))
plt.title("Example Images", fontsize=18)
plt.imshow(grid)
plt.xticks([ls - ls/2, ls*2-ls/2, ls*3 - ls/2], ("Daging Ayam", "Daging Ayam", "Daging Ayam"), fontsize=15)
plt.tick_params(axis=u'both', which=u'both',length=0)
plt.yticks([])
plt.show()

ModuleNotFoundError: No module named 'PIL'

In [5]:
def train_val_datasets(dataset):
    train_size = int(0.8 * len(dataset))
    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size)
    return train_dataset, val_dataset